In [17]:
from datetime import timedelta

import geopandas as gpd
import movingpandas as mpd
import pandas as pd
import pyarrow as pa

from lonboard import Map, PathLayer, TripsLayer, viz
from lonboard.colormap import apply_categorical_cmap
from lonboard.basemap import CartoStyle, MaplibreBasemap

In [2]:
url = "https://raw.githubusercontent.com/movingpandas/movingpandas-examples/refs/heads/main/data/ais.gpkg"
gdf = gpd.read_file(url, use_arrow=True)

In [3]:
gdf.head()

,Timestamp,MMSI,NavStatus,SOG,COG,Name,ShipType,geometry
0,05/07/2017 00:00:03,219632000,Under way using engine,0.0,270.4,None,Undefined,POINT (11.85958 57.68817)
1,05/07/2017 00:00:05,265650970,Under way using engine,0.0,0.5,None,Undefined,POINT (11.84175 57.6615)
2,05/07/2017 00:00:06,265503900,Under way using engine,0.0,0.0,None,Undefined,POINT (11.9065 57.69077)
3,05/07/2017 00:00:14,219632000,Under way using engine,0.0,188.4,None,Undefined,POINT (11.85958 57.68817)
4,05/07/2017 00:00:19,265519650,Under way using engine,0.0,357.2,None,Undefined,POINT (11.87192 57.68233)


In [5]:
viz(gdf)

In [6]:
print(f"Original size: {len(gdf)} rows")
gdf = gdf[gdf["SOG"] > 0]
print(f"Reduced to {len(gdf)} rows after removing 0 speed records")

Original size: 84702 rows
Reduced to 33593 rows after removing 0 speed records


In [7]:
gdf["t"] = pd.to_datetime(gdf["Timestamp"], format="%d/%m/%Y %H:%M:%S")
traj_collection = mpd.TrajectoryCollection(gdf, "MMSI", t="t", min_length=100)
print(f"Finished creating {len(traj_collection)} trajectories")

Finished creating 77 trajectories


In [8]:
traj_collection = mpd.MinTimeDeltaGeneralizer(traj_collection).generalize(
    tolerance=timedelta(minutes=1),
)

In [9]:
shiptype_to_color = {
    "Passenger": "blue",
    "HSC": "green",
    "Tanker": "red",
    "Cargo": "orange",
    "Sailing": "yellow",
    "Other": "grey",
    "Tug": "grey",
    "SAR": "grey",
    "Undefined": "grey",
    "Pleasure": "grey",
    "Dredging": "grey",
    "Law enforcement": "grey",
    "Pilot": "grey",
    "Fishing": "pink",
    "Diving": "grey",
    "Spare 2": "grey",
}

In [10]:
ship_types = []
for traj in traj_collection.trajectories:
    unique_vals = traj.df["ShipType"].unique()
    # Apparently this is not always consistent
    # assert len(unique_vals) == 1, "Expected single ship type per trajectory"
    ship_types.append(unique_vals[0])

In [11]:
get_color = apply_categorical_cmap(pa.array(ship_types), shiptype_to_color)

In [19]:
trips_layer = TripsLayer.from_movingpandas(
    traj_collection,
    get_color=get_color,
    width_min_pixels=5,
    trail_length=300,
)

/usr/local/lib/python3.12/site-packages/lonboard/traits/_timestamp.py:152: UserWarning: Reducing precision of input timestamp data to 's' to fit into available GPU precision.
  warnings.warn(


In [26]:
bmap = MaplibreBasemap(mode='reverse-controlled', style = CartoStyle.DarkMatter)
m = Map(trips_layer, basemap = bmap, height=600)
m

In [27]:
trips_layer.animate(step=timedelta(seconds=30), fps=50)

In [28]:
linestring_layer = PathLayer(
    table=trips_layer.table,
    get_color=get_color,
    width_min_pixels=2,
    opacity=0.01,
)

In [29]:
m.add_layer(linestring_layer)